In [2]:
!pip install langgraph

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.13 requires langchain-core<0.2.0,>=0.1.33, but you have langchain-core 0.3.45 which is incompatible.
langchain-cohere 0.1.5 requires langchain-core<0.3,>=0.1.42, but you have langchain-core 0.3.45 which is incompatible.
langchain-community 0.0.29 requires langchain-core<0.2.0,>=0.1.33, but you have langchain-core 0.3.45 which is incompatible.
langchain-openai 0.1.7 requires langchain-core<0.3,>=0.1.46, but you have langchain-core 0.3.45 which is incompatible.
langchain-text-splitters 0.0.2 requires langchain-core<0.3,>=0.1.28, but you have langchain-core 0.3.45 which is incompatible.



   ---------------------------------------- 0.0/132.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/132.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/132.5 kB ? eta -:--:--
   --- ------------------------------------ 10.2/132.5 kB ? eta -:--:--
   --- ------------------------------------ 10.2/132.5 kB ? eta -:--:--
   --------- ----------------------------- 30.7/132.5 kB 217.9 kB/s eta 0:00:01
   --------- ----------------------------- 30.7/132.5 kB 217.9 kB/s eta 0:00:01
   ------------------ -------------------- 61.4/132.5 kB 233.8 kB/s eta 0:00:01
   ------------------ -------------------- 61.4/132.5 kB 233.8 kB/s eta 0:00:01
   --------------------- ----------------- 71.7/132.5 kB 196.9 kB/s eta 0:00:01
   --------------------- ----------------- 71.7/132.5 kB 196.9 kB/s eta 0:00:01
   --------------------- ----------------- 71.7/132.5 kB 196.9 kB/s eta 0:00:01
   ----------------------------- -------- 102.4/132.5 kB 210.9 kB/s eta 0:00:01
  

In [5]:
#Disable Proxy

import os

def clear_proxy_settings():
    for var in ["HTTP_PROXY", "HTTPS_PROXY", "ALL_PROXY", "http_proxy", "https_proxy", "all_proxy"]:
        if var in os.environ:
            del os.environ[var]

clear_proxy_settings()

In [7]:
# memory.py - using Ollama for model inference

import os
from typing import Dict, List, Any, Optional
import json
from datetime import datetime
import requests  # For Ollama API calls

class Result:
    """Wrapper class to mimic the result object from LangGraph's InMemoryStore"""
    def __init__(self, id, value):
        self.id = id
        self.value = value

class SimpleMemoryStore:
    """A simple in-memory store that doesn't require embeddings or external APIs"""
    
    def __init__(self):
        # Main storage dictionary
        self.storage = {}
    
    def put(self, namespace, key, value):
        """Store data in the specified namespace under the given key"""
        ns_str = self._format_namespace(namespace)
        if ns_str not in self.storage:
            self.storage[ns_str] = {}
        self.storage[ns_str][key] = value
        return True
    
    def get(self, namespace, key):
        """Retrieve data from the specified namespace under the given key"""
        ns_str = self._format_namespace(namespace)
        if ns_str not in self.storage or key not in self.storage[ns_str]:
            return None
        
        return Result(key, self.storage[ns_str][key])
    
    def search(self, namespace, query, filter_func=None):
        """Simple keyword-based search (no embeddings)"""
        ns_str = self._format_namespace(namespace)
        if ns_str not in self.storage:
            return []
        
        results = []
        for key, value in self.storage[ns_str].items():
            # Very basic text search - check if query exists in the key or stringified value
            value_str = json.dumps(value).lower()
            match_found = (query.lower() in key.lower() or 
                          query.lower() in value_str)
            
            if match_found:
                # Apply filter if provided
                result = Result(key, value)
                if filter_func is None or filter_func(result):
                    results.append(result)
        
        return results
    
    def _format_namespace(self, namespace):
        """Convert tuple namespace to string representation"""
        if isinstance(namespace, tuple):
            return "/".join(namespace)
        return str(namespace)


# Initialize our simple store
store = SimpleMemoryStore()


class EpisodicMemory:
    """Stores specific experiences and events"""
    def __init__(self, store, namespace):
        self.store = store
        self.namespace = namespace
    
    def add_interaction(self, interaction_id, content):
        """Store a specific interaction or event"""
        self.store.put(
            self.namespace,
            f"interaction:{interaction_id}",
            {"content": content, "type": "episodic", "timestamp": str(datetime.now())}
        )
        return f"Stored interaction {interaction_id}"
    
    def recall_interaction(self, interaction_id):
        """Retrieve a specific interaction by ID"""
        result = self.store.get(self.namespace, f"interaction:{interaction_id}")
        if result is None:
            return f"No memory found for interaction {interaction_id}"
        return result.value['content']
    
    def search_interactions(self, query):
        """Search for relevant interactions based on content"""
        results = self.store.search(
            self.namespace, 
            query,
            filter_func=lambda x: x.value.get('type') == 'episodic'
        )
        return [r.value['content'] for r in results]


class SemanticMemory:
    """Stores factual knowledge and concepts"""
    def __init__(self, store, namespace):
        self.store = store
        self.namespace = namespace
    
    def add_knowledge(self, key, facts):
        """Store factual knowledge about a concept"""
        self.store.put(
            self.namespace,
            f"concept:{key}",
            {"facts": facts, "type": "semantic"}
        )
        return f"Stored knowledge about {key}"
    
    def get_knowledge(self, key):
        """Retrieve knowledge about a specific concept"""
        result = self.store.get(self.namespace, f"concept:{key}")
        if result is None:
            return f"No knowledge found about {key}"
        return result.value['facts']
    
    def search_knowledge(self, query):
        """Search for relevant knowledge"""
        results = self.store.search(
            self.namespace, 
            query,
            filter_func=lambda x: x.value.get('type') == 'semantic'
        )
        return [(r.id.split(':')[-1], r.value['facts']) for r in results]


class ProceduralMemory:
    """Stores information about how to perform tasks or follow procedures"""
    def __init__(self, store, namespace):
        self.store = store
        self.namespace = namespace
    
    def add_procedure(self, name, instructions):
        """Store a procedure or instructions for a task"""
        self.store.put(
            self.namespace,
            f"procedure:{name}",
            {"instructions": instructions, "type": "procedural"}
        )
        return f"Stored procedure {name}"
    
    def get_procedure(self, name):
        """Retrieve instructions for a specific procedure"""
        result = self.store.get(self.namespace, f"procedure:{name}")
        if result is None:
            return f"No procedure found for {name}"
        return result.value['instructions']
    
    def update_procedure(self, name, new_instructions):
        """Update existing procedure instructions"""
        existing = self.store.get(self.namespace, f"procedure:{name}")
        if existing is None:
            return f"No procedure found for {name}"
        
        self.store.put(
            self.namespace,
            f"procedure:{name}",
            {"instructions": new_instructions, "type": "procedural"}
        )
        return f"Updated procedure {name}"
    
    def search_procedures(self, query):
        """Search for relevant procedures"""
        results = self.store.search(
            self.namespace, 
            query,
            filter_func=lambda x: x.value.get('type') == 'procedural'
        )
        return [(r.id.split(':')[-1], r.value['instructions']) for r in results]


class IntegratedMemory:
    """Combines episodic, semantic, and procedural memory into a single system"""
    def __init__(self, user_id):
        self.user_id = user_id
        self.store = store  # Using the globally defined store
        
        # Initialize all memory types
        self.episodic = EpisodicMemory(store, (user_id, "episodes"))
        self.semantic = SemanticMemory(store, (user_id, "knowledge"))
        self.procedural = ProceduralMemory(store, (user_id, "procedures"))
    
    def query_memory(self, query):
        """Query across all memory types"""
        results = {
            "episodic": self.episodic.search_interactions(query),
            "semantic": self.semantic.search_knowledge(query),
            "procedural": self.procedural.search_procedures(query)
        }
        return results
    
    def remember_interaction(self, interaction_id, content):
        return self.episodic.add_interaction(interaction_id, content)
    
    def learn_fact(self, concept, facts):
        return self.semantic.add_knowledge(concept, facts)
    
    def learn_procedure(self, name, instructions):
        return self.procedural.add_procedure(name, instructions)


class OllamaMemoryAssistant:
    """Assistant using Ollama instead of HuggingFace models"""
    def __init__(self, user_id, model_name="gemma3:12b", base_url="http://localhost:11434"):
        self.memory = IntegratedMemory(user_id)
        self.model_name = model_name
        self.base_url = base_url
    
    def _format_memory_context(self, memory_results):
        context = ["MEMORY CONTEXT:"]
        
        if memory_results["episodic"]:
            context.append("\nPast experiences:")
            for memory in memory_results["episodic"][:3]:  # Limit to top 3
                context.append(f"- {memory}")
        
        if memory_results["semantic"]:
            context.append("\nKnown facts:")
            for concept, facts in memory_results["semantic"][:3]:  # Limit to top 3
                context.append(f"- {concept}: {facts}")
        
        if memory_results["procedural"]:
            context.append("\nRelevant procedures:")
            for name, instructions in memory_results["procedural"][:3]:  # Limit to top 3
                context.append(f"- {name}: {instructions}")
        
        return "\n".join(context)

    def process_query(self, query):
        """Process a query using Ollama API"""
        # Search memory
        memory_results = self.memory.query_memory(query)
        memory_context = self._format_memory_context(memory_results)
        
        # Create prompt with memory context
        prompt = f"""You are an assistant with access to the user's memories.
Use the following memory context to answer the query:

{memory_context}

User query: {query}"""
        
        # Call Ollama API
        response = requests.post(
            f"{self.base_url}/api/generate",
            json={
                "model": self.model_name,
                "prompt": prompt,
                "stream": False
            }
        )
        
        if response.status_code == 200:
            result = response.json()
            generated_text = result.get("response", "")
        else:
            generated_text = f"Error: Unable to get response from Ollama (Status code: {response.status_code})"
        
        # Store this interaction in episodic memory
        interaction_id = f"query-{hash(query)}"
        self.memory.remember_interaction(interaction_id, {
            "query": query,
            "response": generated_text,
            "timestamp": str(datetime.now())
        })
        
        return generated_text
    
    def learn_fact(self, concept, facts):
        return self.memory.learn_fact(concept, facts)
    
    def learn_procedure(self, name, instructions):
        return self.memory.learn_procedure(name, instructions)
    
    def remember_interaction(self, interaction_id, content):
        return self.memory.remember_interaction(interaction_id, content)


# Example usage
if __name__ == "__main__":
    # Create a memory-enhanced assistant using Ollama
    assistant = OllamaMemoryAssistant("user789", model_name="gemma3:12b")
    
    # Pre-load some memories
    assistant.learn_fact("project_beta", {
        "description": "Mobile payment processing system",
        "status": "In development",
        "deadline": "Q2 2024",
        "team": ["John", "Sarah", "Michael"]
    })
    
    assistant.learn_procedure("handle_client_inquiry", {
        "steps": [
            "1. Acknowledge receipt within 2 hours",
            "2. Determine inquiry category (technical, billing, feature request)",
            "3. Route to appropriate department or handle directly",
            "4. Follow up within 24 hours with status update"
        ]
    })
    
    assistant.remember_interaction("team-standup-2023-11-05", {
        "event": "Daily standup",
        "updates": {
            "John": "Working on API integration",
            "Sarah": "Finished UI redesign",
            "Michael": "Debugging payment gateway issues"
        },
        "blockers": ["Waiting for credentials from payment provider"]
    })
    
    # Example queries
    queries = [
        "What's the current status of Project Beta?",
        "How should I handle a new client inquiry?",
        "What was Michael working on in our last standup?"
    ]
    
    for query in queries:
        print(f"\nQuery: {query}")
        response = assistant.process_query(query)
        print(f"Response: {response}")


Query: What's the current status of Project Beta?
Response: Error: Unable to get response from Ollama (Status code: 500)

Query: How should I handle a new client inquiry?
Response: Error: Unable to get response from Ollama (Status code: 500)

Query: What was Michael working on in our last standup?
Response: Error: Unable to get response from Ollama (Status code: 500)
